<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20251812%5DComplete%20training%20evaluation%20script%20with%20CLEAN%20ABLATION%20(LR%2C%20MLP%2C%20LR%E2%86%92MLP%20uncal).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -*- coding: utf-8 -*-
"""
Complete training/evaluation script with CLEAN ABLATION (LR, MLP, LR→MLP uncal./Temp/Isot.), SEED-AVERAGED.
This file merges your provided code and appends the ablation block + main() call to generate:
  - paper_exports/clean_ablation_seedavg.csv
  - paper_exports/clean_ablation_seedavg.tex

Publication-safe CIC protocol (Option A): strict Train/Val/Test split with validation-only calibration.

In [ ]:
import os, json, argparse, re, difflib, traceback, zipfile, time
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline as SkPipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, f1_score, classification_report,
                             average_precision_score, roc_auc_score, precision_recall_curve,
                             confusion_matrix, ConfusionMatrixDisplay, log_loss)
from sklearn.isotonic import IsotonicRegression
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold
from joblib import dump, load

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

In [ ]:
# -----------------------------
#          Config
# -----------------------------
CFG = {
    "paths": {
        "nf_csv": "/content/Dataset_NF-ToN-IoT.csv",
        "cic_train_csv": "/content/CIC_IoMT_2024_WiFi_MQTT_train.csv",
        "cic_test_csv": "/content/CIC_IoMT_2024_WiFi_MQTT_test.csv",
        "outdir": "/mnt/data/iot_ids_refactor/outputs",
        "run_dir": None
},
    "label_columns": {
        "binary_candidates": ["Label", "label", "Binary", "binary"],
        "multiclass_candidates": ["Class", "class", "Category", "category"],
        "drop_non_feature_if_present": ["Attack", "attack", "Label", "label", "Class", "class"]
    },
    "train": {
        "random_state": 42,
        "test_size": 0.2,
        "use_smote": True,
        "mlp_hidden_units": 64,
        "max_epochs": 25,
        "batch_size": 2048
    },
    "metrics": {"target_drs": [0.90, 0.95]},
    "calibration": {"cic_calib_frac": 0.10},
    "robust": {"eps": [0.05, 0.10], "pgd_steps": 10, "pgd_alpha": 0.02},
    "automap": {"similarity_threshold": 0.75, "max_pairs": 256},
    "cic_optionA": {
        "train_frac": 0.60,
        "val_frac": 0.20,
        "test_frac": 0.20,
        "val_cal_frac": 0.50,
        "grid_C": [0.5, 1.0, 2.0],
        "grid_hidden": [32, 64, 128]
    }
}

PERF = {"adv_eval_max_rows": 200_000, "plot_max_rows": 150_000}

In [ ]:
# ------------------------------------------
#        Canonical feature aliases
# ------------------------------------------
CANON = {
    "duration": ["dur","flow_duration","duration","dur_ms","flowdur","flow_dur","Flow Duration"],
    "tot_fwd_pkts": ["Tot Fwd Pkts","tot_fwd_pkts","total_fwd_packets","fwd_pkts_tot","fwd_pkts_total","fwd_packets_total"],
    "tot_bwd_pkts": ["Tot Bwd Pkts","tot_bwd_pkts","total_bwd_packets","bwd_pkts_tot","bwd_pkts_total","bwd_packets_total"],
    "totlen_fwd_pkts": ["TotLen Fwd Pkts","totlen_fwd_pkts","total_length_of_fwd_packets","fwd_pkts_len_tot","fwd_bytes_total","total_fwd_bytes","Tot Fwd Bytes","Fwd Bytes"],
    "totlen_bwd_pkts": ["TotLen Bwd Pkts","totlen_bwd_pkts","total_length_of_bwd_packets","bwd_pkts_len_tot","bwd_bytes_total","total_bwd_bytes","Tot Bwd Bytes","Bwd Bytes"],
    "fwd_pkt_len_mean": ["Fwd Pkt Len Mean","fwd_pkt_len_mean","fwd_packet_length_mean","fwd_pkt_length_mean","Fwd Pkt Len Avg"],
    "bwd_pkt_len_mean": ["Bwd Pkt Len Mean","bwd_pkt_len_mean","bwd_packet_length_mean","bwd_pkt_length_mean","Bwd Pkt Len Avg"],
    "fwd_iat_mean": ["Fwd IAT Mean","fwd_iat_mean","fwd_interarrival_mean","fwd_iat_avg","Fwd IAT Avg"],
    "bwd_iat_mean": ["Bwd IAT Mean","bwd_iat_mean","bwd_interarrival_mean","bwd_iat_avg","Bwd IAT Avg"],
    "pkt_len_mean": ["Pkt Len Mean","pkt_len_mean","packet_length_mean","pkt_length_mean","Pkt Len Avg","Average Packet Size"],
    "pkt_len_std": ["Pkt Len Std","pkt_len_std","packet_length_std","pkt_length_std","Std Packet Len","Pkt Len STD"],
    "flow_pkts_s": ["Flow Pkts/s","flow_pkts_s","packets_per_second","pkts_per_sec","pkt_rate","Packet Rate","Pkts/s"],
    "flow_byts_s": ["Flow Byts/s","flow_byts_s","bytes_per_second","byte_rate","bytes_per_sec","throughput","Bps","Bytes/s"],
    "protocol": ["Protocol","proto","protocol","protocol_type","l4_proto","Protocol Type"],
    "src_port": ["Src Port","src_port","sport","source_port","Source Port"],
    "dst_port": ["Dst Port","dst_port","dport","destination_port","Destination Port"],
    "flags": ["flags","tcp_flags","flag_count","tcpflag","flag_total","TCP Flags","Flags"]
}

In [ ]:
# -------------------------
#     Feature utilities
# -------------------------
def build_rename_map(df_cols):
    lower = {c.lower(): c for c in df_cols}
    rename = {}
    for canon, aliases in CANON.items():
        for a in aliases:
            key = a.lower()
            if key in lower:
                rename[lower[key]] = canon
                break
    return rename

def normalize_features(df):
    rename = build_rename_map(df.columns)
    return df.rename(columns=rename)

def add_derived_flow_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    dur_cols = ["flow_duration", "Flow Duration", "duration", "dur", "dur_ms", "flow_dur"]
    dur = None
    for c in dur_cols:
        if c in d.columns:
            dur = pd.to_numeric(d[c], errors="coerce"); break
    if dur is None:
        d["flow_dur_s"] = 1.0
    else:
        perc95 = np.nanquantile(dur, 0.95)
        d["flow_dur_s"] = np.where(perc95 > 100.0, dur / 1000.0, dur)
    def find(colnames):
        for c in colnames:
            if c in d.columns: return pd.to_numeric(d[c], errors="coerce")
        return None
    fwd_pkts = find(["tot_fwd_pkts","Tot Fwd Pkts","total_fwd_packets","fwd_pkts_total","fwd_packets_total"])
    bwd_pkts = find(["tot_bwd_pkts","Tot Bwd Pkts","total_bwd_packets","bwd_pkts_tot","bwd_pkts_total","bwd_packets_total"])
    fwd_byts = find(["totlen_fwd_pkts","TotLen Fwd Pkts","total_fwd_bytes","Fwd Bytes","Tot Fwd Bytes"])
    bwd_byts = find(["totlen_bwd_pkts","TotLen Bwd Pkts","total_bwd_bytes","Bwd Bytes","Tot Bwd Bytes"])
    d["tot_pkts"] = 0.0
    if fwd_pkts is not None: d["tot_pkts"] = d["tot_pkts"] + fwd_pkts.fillna(0)
    if bwd_pkts is not None: d["tot_pkts"] = d["tot_pkts"] + bwd_pkts.fillna(0)
    d["tot_byts"] = 0.0
    if fwd_byts is not None: d["tot_byts"] = d["tot_byts"] + fwd_byts.fillna(0)
    if bwd_byts is not None: d["tot_byts"] = d["tot_byts"] + bwd_byts.fillna(0)
    safe_dur = np.maximum(d["flow_dur_s"].replace(0, np.nan), 1e-3)
    safe_pkts = np.maximum(d["tot_pkts"].replace(0, np.nan), 1.0)
    d["pkts_per_sec"] = d["tot_pkts"] / safe_dur
    d["byts_per_sec"] = d["tot_byts"] / safe_dur
    d["avg_pkt_len"]  = d["tot_byts"] / safe_pkts
    for pcol, newcol in [("Src Port","src_port"),("src_port","src_port"),("sport","src_port"),
                         ("Dst Port","dst_port"),("dst_port","dst_port"),("dport","dst_port")]:
        if pcol in d.columns:
            d[newcol] = pd.to_numeric(d[pcol], errors="coerce")
    proto_col = None
    for c in ["protocol","Protocol","protocol_type","l4_proto"]:
        if c in d.columns: proto_col = c; break
    d["is_tcp"] = 0.0; d["is_udp"] = 0.0
    if proto_col is not None:
        pc = d[proto_col]
        if np.issubdtype(pc.dtype, np.number):
            d["is_tcp"] = (pc == 6).astype(float)
            d["is_udp"] = (pc == 17).astype(float)
        else:
            pstr = pc.astype(str).str.lower()
            d["is_tcp"] = pstr.str.contains("tcp").astype(float)
            d["is_udp"] = pstr.str.contains("udp").astype(float)
    return d

In [ ]:
# -------------------
#     I/O helpers
# -------------------
def detect_label_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def pick_features(df, drop_cols):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    feat_cols = [c for c in num_cols if c not in set(drop_cols)]
    return feat_cols

def load_dataset(path):
    df_raw = pd.read_csv(path)
    df = normalize_features(df_raw)
    df = add_derived_flow_features(df)
    bin_col = detect_label_column(df, CFG["label_columns"]["binary_candidates"])
    mc_col  = detect_label_column(df, CFG["label_columns"]["multiclass_candidates"])
    feat_cols = pick_features(df, CFG["label_columns"]["drop_non_feature_if_present"])
    return df, feat_cols, bin_col, mc_col

In [ ]:
# -----------------
#      Labels
# -----------------
def map_label_string_to_family(s: str) -> str:
    st = str(s).lower().strip()
    if "benign" in st or "normal" in st: return "Benign"
    return "Attack"

def build_binary_labels(df, bin_col, mc_col):
    if bin_col and bin_col in df.columns:
        try:
            y_bin = df[bin_col].astype(int).values
        except ValueError:
            y_bin = df[bin_col].apply(lambda x: 0 if map_label_string_to_family(str(x)) == "Benign" else 1).values
    elif mc_col and mc_col in df.columns:
        try:
            y_bin = (df[mc_col].astype(int).values != 0).astype(int)
        except ValueError:
            y_bin = df[mc_col].apply(lambda x: 0 if map_label_string_to_family(str(x)) == "Benign" else 1).values
    else:
        raise ValueError("No binary or multiclass label column found.")
    return y_bin

def make_one_hot(y, n_classes):
    out = np.zeros((len(y), n_classes), dtype=int)
    out[np.arange(len(y)), y] = 1
    return out

In [ ]:
# -------------------
#     Calibration
# -------------------
def sigmoid(x): return 1.0/(1.0+np.exp(-x))

def fit_temperature(z_val, y_val, n_iters=300, lr=0.05):
    T = 1.0
    for _ in range(n_iters):
        p = sigmoid(z_val / T)
        p = np.clip(p, 1e-7, 1-1e-7)
        grad = np.mean((p - y_val) * (-z_val/(T*T)))
        T -= lr * grad
        T = float(np.clip(T, 0.05, 50.0))
    return T

def apply_temperature(z, T):
    return sigmoid(z / T)

def calibrate_scores(scores_cal, y_cal_bin, scores_eval, method="temperature"):
    s_cal = np.clip(scores_cal, 1e-6, 1-1e-6)
    s_eval = np.clip(scores_eval, 1e-6, 1-1e-6)
    if method == "isotonic":
        iso = IsotonicRegression(out_of_bounds="clip")
        iso.fit(s_cal, y_cal_bin.astype(int))
        s_eval_cal = iso.predict(s_eval)
        return s_eval_cal, {"method": "isotonic"}
    else:
        z_cal = np.log(s_cal/(1-s_cal))
        T = fit_temperature(z_cal, y_cal_bin.astype(int), n_iters=300, lr=0.05)
        z_eval = np.log(s_eval/(1-s_eval))
        s_eval_cal = apply_temperature(z_eval, T)
        return s_eval_cal, {"method": "temperature", "T": float(T)}

In [ ]:
# ------------------------
#     Metrics & plots
# ------------------------
def fpr_at_dr(y_true, scores, target_dr=0.95, positive_label=1):
    thresholds = np.unique(scores)
    thresholds.sort()
    best_fpr = None
    best_thr = None
    for thr in thresholds:
        y_pred = (scores >= thr).astype(int)
        pos = (y_true == positive_label)
        neg = ~pos
        tp = (pos & (y_pred == 1)).sum()
        fn = (pos & (y_pred == 0)).sum()
        dr = tp / max(tp + fn, 1)
        if dr >= target_dr:
            fp = (neg & (y_pred == 1)).sum()
            fpr = fp / max(neg.sum(), 1)
            if best_fpr is None or fpr < best_fpr:
                best_fpr = fpr; best_thr = thr
    return best_fpr if best_fpr is not None else np.nan, best_thr

def expected_calibration_error(y_true, probas, n_bins=15):
    confidences = probas.max(axis=1)
    predictions = probas.argmax(axis=1)
    correct = (predictions == y_true).astype(float)
    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if mask.sum() == 0: continue
        acc = correct[mask].mean()
        conf = confidences[mask].mean()
        ece += (mask.mean()) * abs(acc - conf)
    return ece

def plot_pr_curves(y_onehot, probas, class_names, out_png):
    if y_onehot.shape[0] > 150000:
        idx = np.random.RandomState(42).choice(y_onehot.shape[0], size=150000, replace=False)
        y_onehot = y_onehot[idx]; probas = probas[idx]
    n_classes = y_onehot.shape[1]
    plt.figure()
    for c in range(n_classes):
        precision, recall, _ = precision_recall_curve(y_onehot[:, c], probas[:, c])
        ap = average_precision_score(y_onehot[:, c], probas[:, c])
        plt.plot(recall, precision, label=f"{class_names[c]} (AP={ap:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall Curves")
    plt.legend(); plt.tight_layout(); plt.savefig(out_png, dpi=180); plt.close()

def plot_confusion(y_true, y_pred, class_names, out_png, normalize='true'):
    if y_true.shape[0] > 150000:
        idx = np.random.RandomState(42).choice(y_true.shape[0], size=150000, replace=False)
        y_true = y_true[idx]; y_pred = y_pred[idx]
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)), normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(); disp.plot(ax=ax, values_format=".2f", cmap=None, colorbar=False)
    plt.title("Confusion Matrix" + (f" (normalized={normalize})" if normalize else ""))
    plt.tight_layout(); plt.savefig(out_png, dpi=180); plt.close()

In [ ]:
# ----------------------------------------------------
#          Robust, NaN-safe preprocessing
# ----------------------------------------------------
class SafeNaNDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        X = np.where(np.isfinite(X), X, np.nan)
        self.keep_mask_ = ~np.all(np.isnan(X), axis=0)
        if not np.any(self.keep_mask_):
            raise ValueError("All features are NaN after cleaning.")
        return self
    def transform(self, X):
        X = np.asarray(X, dtype=float)
        X = np.where(np.isfinite(X), X, np.nan)
        return X[:, self.keep_mask_]

In [ ]:
# -----------------
#     Models
# -----------------
def build_scaler(name):
    if name == "standard":
        return StandardScaler()
    elif name == "robust":
        return RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))
    elif name == "quantile":
        return QuantileTransformer(output_distribution="normal", subsample=200000, random_state=42)
    else:
        return StandardScaler()

def get_lr_pipeline(use_smote, random_state, scaler_name='standard', C=1.0):
    lr = LogisticRegression(C=C, max_iter=2000, solver="lbfgs", class_weight="balanced", n_jobs=-1)
    steps = [
        ("clean", SafeNaNDropper()),
        ("imputer", SimpleImputer(strategy='median', add_indicator=True)),
        ("varth", VarianceThreshold(threshold=0.0)),
        ("scaler", build_scaler(scaler_name)),
    ]
    if use_smote:
        steps.append(("smote", SMOTE(random_state=random_state)))
    steps.append(("lr", lr))
    pipe = ImbPipeline(steps=steps)
    return pipe

def fit_lr_then_mlp(X_train, y_train, X_val, y_val, use_smote, random_state, max_epochs, batch_size, hidden_units, alpha=1e-4, scaler_name='standard', C=1.0, early_stopping=True):
    if X_train.shape[1] != X_val.shape[1]:
        print(f"[ERROR] Feature mismatch: X_train has {X_train.shape[1]} features, X_val has {X_val.shape[1]}.")
        return None, None
    if len(np.unique(y_train)) < 2:
        print("[WARN] Skipping LR pipeline (single-class training data).")
        return None, None
    pipe = get_lr_pipeline(use_smote, random_state, scaler_name=scaler_name, C=C)
    pipe.fit(X_train, y_train)
    try:
        Z_train = pipe.decision_function(X_train).reshape(-1, 1)
        Z_val = pipe.decision_function(X_val).reshape(-1, 1)
    except Exception:
        Z_train = np.log(np.clip(pipe.predict_proba(X_train), 1e-7, 1-1e-7))
        Z_val = np.log(np.clip(pipe.predict_proba(X_val), 1e-7, 1-1e-7))
        if Z_train.ndim == 1: Z_train = Z_train.reshape(-1, 1)
        if Z_val.ndim   == 1: Z_val   = Z_val.reshape(-1, 1)
    mlp = MLPClassifier(hidden_layer_sizes=(hidden_units,), alpha=alpha,
                        batch_size=batch_size, learning_rate_init=1e-3,
                        max_iter=max_epochs, random_state=random_state,
                        early_stopping=True, n_iter_no_change=5, validation_fraction=0.1)
    mlp.fit(Z_train, y_train)
    return pipe, mlp

In [ ]:
# ------------------------------------------------
#        Binary / Multiclass evaluation
# ------------------------------------------------
def eval_binary(y_true, prob_pos, tag, outdir):
    if prob_pos is None or len(prob_pos) == 0:
        with open(os.path.join(outdir, f"{tag}__binary_report.txt"), "w") as f:
            f.write("Binary evaluation skipped due to insufficient classes or probabilities.\n")
        return
    auc = roc_auc_score(y_true, prob_pos)
    aupr = average_precision_score(y_true, prob_pos)
    metrics = {"roc_auc": float(auc), "aupr": float(aupr)}
    for dr in [0.90, 0.95]:
        fpr, thr = fpr_at_dr(y_true, prob_pos, target_dr=dr)
        metrics[f"fpr@dr={dr}"] = float(fpr) if fpr==fpr else None
        metrics[f"thr@dr={dr}"] = float(thr) if thr is not None else None
    os.makedirs(outdir, exist_ok=True)
    with open(os.path.join(outdir, f"{tag}__binary_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=2)
    best_thr = metrics.get("thr@dr=0.95", 0.5)
    y_pred = (prob_pos >= (best_thr if best_thr is not None else 0.5)).astype(int)
    report = classification_report(y_true, y_pred, digits=4)
    with open(os.path.join(outdir, f"{tag}__binary_report.txt"), "w") as f:
        f.write(report)

def eval_multiclass(y_true, probas, class_names, tag, outdir, train_classes=None):
    if probas is None or probas.shape[0] == 0:
        with open(os.path.join(outdir, f"{tag}__multiclass_metrics.json"), "w") as f:
            json.dump({"accuracy": np.nan, "f1_macro": np.nan,
                       "aupr_micro": np.nan, "aupr_per_class": {}, "ece": np.nan}, f, indent=2)
        return
    test_classes = sorted(np.unique(y_true))
    if train_classes is None:
        k = min(len(test_classes), probas.shape[1])
        probas_aligned = np.zeros((probas.shape[0], len(test_classes)))
        probas_aligned[:, :k] = probas[:, :k]
        used_classes = test_classes
    else:
        used_classes = test_classes
        probas_aligned = np.zeros((probas.shape[0], len(test_classes)))
        for j, cls in enumerate(train_classes):
            if cls in test_classes:
                idx = test_classes.index(cls)
                probas_aligned[:, idx] = probas[:, j]
    y_pred = probas_aligned.argmax(axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    class_names_eval = [f"C{c}" for c in used_classes]
    y_idx = np.array([used_classes.index(c) for c in y_true])
    y_onehot = make_one_hot(y_idx, len(used_classes))
    aupr_micro = average_precision_score(y_onehot, probas_aligned, average="micro")
    aupr_per = {class_names_eval[i]: float(average_precision_score(y_onehot[:, i], probas_aligned[:, i]))
                for i in range(len(used_classes))}
    ece = expected_calibration_error(y_idx, probas_aligned, n_bins=15)
    with open(os.path.join(outdir, f"{tag}__multiclass_metrics.json"), "w") as f:
        json.dump({"accuracy": float(acc), "f1_macro": float(f1m),
                   "aupr_micro": float(aupr_micro),
                   "aupr_per_class": aupr_per, "ece": float(ece)}, f, indent=2)
    plot_pr_curves(y_onehot, probas_aligned, class_names_eval, os.path.join(outdir, f"{tag}__pr_curves.png"))
    plot_confusion(y_idx, y_pred, class_names_eval, os.path.join(outdir, f"{tag}__confusion.png"))

In [ ]:
# --------------------------------------------
#            Adversarial utils
# --------------------------------------------
def fgsm(X_std, y, model, eps):
    W = model.coef_.reshape(1, -1)
    z = model.decision_function(X_std)
    p = 1.0/(1.0+np.exp(-z))
    grad = (p - y.reshape(-1))[:, None] * W
    return X_std + eps * np.sign(grad)

def pgd(X_std, y, model, eps, alpha, steps):
    X_adv = X_std.copy()
    for _ in range(steps):
        X_adv = fgsm(X_adv, y, model, alpha)
        delta = np.clip(X_adv - X_std, -eps, eps)
        X_adv = X_std + delta
    return X_adv

def clip_to_train_range(X_std, scaler, X_train_std_min, X_train_std_max):
    X_raw = X_std
    X_raw = np.clip(X_raw, X_train_std_min, X_train_std_max)
    return X_raw

In [ ]:
# ----------------------
#       Automapper
# ----------------------
_token_map = {
    "packets":"pkts","packet":"pkt","length":"len","average":"mean","stddev":"std","std_dev":"std",
    "backward":"bwd","forward":"fwd","persec":"persec","per_second":"persec","rate":"persec",
    "bytes":"byts","byte":"byt","duration":"dur","interarrival":"iat","src":"src","dst":"dst",
    "throughput":"bytspersec","bps":"bytspersec","bpss":"bytspersec"
}
def keyify(name: str) -> str:
    s = re.sub(r'[^a-zA-Z0-9]+', '', name.lower())
    for k,v in _token_map.items():
        s = s.replace(k, v)
    s = re.sub(r'[aeiou]', '', s)
    return s

def automap_features(dfA, featsA, dfB, featsB, threshold=0.86, max_pairs=64):
    keysA = {f: keyify(f) for f in featsA}
    keysB = {f: keyify(f) for f in featsB}
    pairs = []
    for a,ka in keysA.items():
        best_b = None; best_sim = 0.0
        for b,kb in keysB.items():
            sim = difflib.SequenceMatcher(None, ka, kb).ratio()
            if sim > best_sim: best_sim, best_b = sim, b
        if best_sim >= threshold and np.issubdtype(dfA[a].dtype, np.number) and np.issubdtype(dfB[best_b].dtype, np.number):
            pairs.append((a, b, float(sim)))
    pairs.sort(key=lambda x: -x[2])
    usedA, usedB, final = set(), set(), []
    for a,b,sim in pairs:
        if a in usedA or b in usedB: continue
        final.append((a,b,sim))
        usedA.add(a); usedB.add(b)
        if len(final) >= max_pairs: break
    return final

def apply_automap_and_rename(df_src, feats_src, df_tgt, feats_tgt, outdir, tag_prefix):
    common = list(sorted(set(feats_src).intersection(set(feats_tgt))))
    audit = {"mode": "intersection", "count": len(common), "pairs": []}
    audit_path = os.path.join(outdir, f"{tag_prefix}__feature_automap.json")
    if len(common) > 0:
        with open(audit_path, "w") as f: json.dump(audit, f, indent=2)
        with open(os.path.join(outdir, f"{tag_prefix}_common_features.json"), "w") as f:
            json.dump({"count": len(common), "features": common, "audit": os.path.basename(audit_path)}, f, indent=2)
        return df_src, df_tgt, common, audit_path
    matches = automap_features(df_src, feats_src, df_tgt, feats_tgt,
                               threshold=CFG["automap"]["similarity_threshold"],
                               max_pairs=CFG["automap"]["max_pairs"])
    audit = {"mode": "automap", "count": len(matches),
             "pairs": [{"src":a,"tgt":b,"similarity":sim} for a,b,sim in matches]}
    df_tgt2 = df_tgt.copy()
    rename_map = {b:a for a,b,_ in matches}
    df_tgt2 = df_tgt2.rename(columns=rename_map)
    common2 = [a for a,_,_ in matches]
    audit["renamed"] = rename_map
    with open(audit_path, "w") as f: json.dump(audit, f, indent=2)
    with open(os.path.join(outdir, f"{tag_prefix}_common_features.json"), "w") as f:
        json.dump({"count": len(common2), "features": common2, "audit": os.path.basename(audit_path)}, f, indent=2)
    return df_src, df_tgt2, common2, audit_path

In [ ]:
# --------------------------------------------------
#         Helpers: RUN_DIR + artifact I/O
# --------------------------------------------------
def get_run_dir(outdir):
    rd = CFG["paths"]["run_dir"] or os.path.join(outdir, "artifacts")
    os.makedirs(rd, exist_ok=True)
    return rd

def save_artifact(obj, path):
    tmp = path + f".tmp{int(time.time()*1000)}"
    dump(obj, tmp)
    os.replace(tmp, path)

def metric_row_csv(csv_path, header, row):
    new = not os.path.exists(csv_path)
    with open(csv_path, "a") as f:
        if new: f.write(",".join(header)+"\n")
        f.write(",".join(map(str,row))+"\n")

In [ ]:
# -----------------------
#      NF in-domain
# -----------------------
def run_in_domain_nf(nf_df, nf_feats, nf_bin_col, nf_mc_col, outdir):
    RUN_DIR = get_run_dir(outdir)

    yb = build_binary_labels(nf_df, nf_bin_col, nf_mc_col)
    X = nf_df[nf_feats].values
    X_tr, X_te, y_tr, y_te = train_test_split(X, yb, test_size=CFG["train"]["test_size"],
                                              stratify=yb, random_state=CFG["train"]["random_state"])

    pipe, mlp = fit_lr_then_mlp(X_tr, y_tr, X_te, y_te,
                                CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                CFG["train"]["mlp_hidden_units"])

    if pipe is not None and mlp is not None:
        save_artifact(pipe, os.path.join(RUN_DIR, "NF_in_domain_pipe.joblib"))
        save_artifact(mlp,  os.path.join(RUN_DIR, "NF_in_domain_mlp.joblib"))

    prob_pos = None
    if pipe is not None:
        try:
            Z_te = pipe.decision_function(X_te).reshape(-1,1)
        except Exception:
            Z_te = np.log(np.clip(pipe.predict_proba(X_te), 1e-7, 1-1e-7)).reshape(-1,1)
        prob_pos = mlp.predict_proba(Z_te)[:,1] if mlp is not None else pipe.predict_proba(X_te)[:,1]
    eval_binary(y_te, prob_pos, "NF_ToN_IoT__in_domain", outdir)

    if prob_pos is not None:
        roc = roc_auc_score(y_te, prob_pos)
        aupr = average_precision_score(y_te, prob_pos)
        f90, _ = fpr_at_dr(y_te, prob_pos, 0.90); f95, _ = fpr_at_dr(y_te, prob_pos, 0.95)
        metric_row_csv(os.path.join(RUN_DIR, "metrics_log.csv"),
                       ["phase","roc_auc","aupr","fpr@0.90","fpr@0.95"],
                       ["NF_clean", roc, aupr, f90, f95])

    if pipe is not None:
        Xt_tr_std = pipe.named_steps["scaler"].transform(
            pipe.named_steps["varth"].transform(
                pipe.named_steps["imputer"].transform(
                    pipe.named_steps["clean"].transform(X_tr))))
        Xt_te_std = pipe.named_steps["scaler"].transform(
            pipe.named_steps["varth"].transform(
                pipe.named_steps["imputer"].transform(
                    pipe.named_steps["clean"].transform(X_te))))
        Xmin, Xmax = Xt_tr_std.min(axis=0), Xt_tr_std.max(axis=0)

        if Xt_te_std.shape[0] > 200000:
            rs = np.random.RandomState(42)
            idx = rs.choice(Xt_te_std.shape[0], size=200000, replace=False)
            Xt_te_std = Xt_te_std[idx]; y_te_adv = y_te[idx]
        else:
            y_te_adv = y_te

        lr = pipe.named_steps["lr"]
        for eps in CFG["robust"]["eps"]:
            X_fgsm = fgsm(Xt_te_std, y_te_adv, lr, eps)
            X_fgsm = clip_to_train_range(X_fgsm, pipe.named_steps["scaler"], Xmin, Xmax)
            z_fgsm = lr.decision_function(X_fgsm).reshape(-1,1)
            prob_fgsm = mlp.predict_proba(z_fgsm)[:,1] if mlp is not None else 1.0/(1.0+np.exp(-z_fgsm)).ravel()
            eval_binary(y_te_adv, prob_fgsm, f"NF_ToN_IoT__in_domain__FGSM_eps={eps}", outdir)

            X_pgd = pgd(Xt_te_std, y_te_adv, lr, eps, CFG["robust"]["pgd_alpha"], CFG["robust"]["pgd_steps"])
            X_pgd = clip_to_train_range(X_pgd, pipe.named_steps["scaler"], Xmin, Xmax)
            z_pgd = lr.decision_function(X_pgd).reshape(-1,1)
            prob_pgd = mlp.predict_proba(z_pgd)[:,1] if mlp is not None else 1.0/(1.0+np.exp(-z_pgd)).ravel()
            eval_binary(y_te_adv, prob_pgd, f"NF_ToN_IoT__in_domain__PGD_eps={eps}", outdir)

    if nf_mc_col:
        ym = nf_df[nf_mc_col].astype(int).values
        Xm_tr, Xm_te, ym_tr, ym_te = train_test_split(X, ym, test_size=CFG["train"]["test_size"],
                                                      stratify=ym, random_state=CFG["train"]["random_state"])
        if len(np.unique(ym_tr)) >= 2:
            pipe_m = SkPipeline([
              ("clean", SafeNaNDropper()),
              ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
              ("scaler", StandardScaler()),
              ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))
            ])
            pipe_m.fit(Xm_tr, ym_tr)
            probas = pipe_m.predict_proba(Xm_te)

            class_names = [f"C{c}" for c in sorted(np.unique(ym))]
            eval_multiclass(ym_te, probas, class_names, "NF_ToN_IoT__in_domain_native_mc", outdir,
                            train_classes=list(pipe_m.named_steps["clf"].classes_))

In [ ]:
# -----------------------------------
# CIC native multiclass
# -----------------------------------
def run_in_domain_cic_native_mc(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir):
    common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
    if len(common) == 0: return
    Xtr = cic_tr_df[common].values
    Xte = cic_te_df[common].values
    ym_tr = cic_tr_df[cic_tr_mc_col].astype(int).values if cic_tr_mc_col else None
    ym_te = cic_te_df[cic_te_mc_col].astype(int).values if cic_te_mc_col else None
    if ym_tr is None or ym_te is None or len(np.unique(ym_tr)) < 2: return

    pipe_m = SkPipeline([
    ("clean", SafeNaNDropper()),
    ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))
    ])
    pipe_m.fit(Xtr, ym_tr)
    probas = pipe_m.predict_proba(Xte)

    class_names = [f"C{c}" for c in sorted(np.unique(ym_te))]
    train_classes = list(pipe_m.named_steps["clf"].classes_)
    eval_multiclass(ym_te, probas, class_names, "CIC_IoMT__native_multiclass__train_to_test", outdir, train_classes=train_classes)

In [ ]:
# --------------------------------------------------------------------------------
#        CIC tiny benign slice experiment (with artifacts + robustness)
# --------------------------------------------------------------------------------
def run_cic_with_tiny_benign_slice(cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                                   cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                                   outdir, slice_frac=0.015, seed=42):
    RUN_DIR = get_run_dir(outdir)
    os.makedirs(outdir, exist_ok=True)
    try:
        rs = np.random.RandomState(seed)
        common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
        if len(common) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: no shared numeric features between CIC_train and CIC_test.\n")
            return
        if cic_tr_mc_col is None or cic_te_mc_col is None:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: multiclass label column not found in CIC train/test.\n")
            return

        y_tr_mc = cic_tr_df[cic_tr_mc_col].astype(int).values
        X_tr_all = cic_tr_df[common].values
        A_idx = np.where(y_tr_mc != 0)[0]
        if len(A_idx) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: CIC_train has no attacks; cannot build binary head.\n")
            return
        X_attack_all = X_tr_all[A_idx]

        y_te_mc = cic_te_df[cic_te_mc_col].astype(int).values
        X_te_all = cic_te_df[common].values
        ben_idx_all = np.where(y_te_mc == 0)[0]
        atk_idx_all = np.where(y_te_mc != 0)[0]
        if len(ben_idx_all) == 0:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: no benign samples in CIC_test.\n")
            return

        n_ben_slice = max(1, int(len(ben_idx_all) * slice_frac))
        ben_slice = rs.choice(ben_idx_all, size=n_ben_slice, replace=False)
        n_ben_cal  = max(1, n_ben_slice // 2)
        n_ben_train = n_ben_slice - n_ben_cal
        rs.shuffle(ben_slice)
        ben_train_idx = ben_slice[:n_ben_train]
        ben_cal_idx   = ben_slice[n_ben_train:]
        n_atk_cal = min(len(atk_idx_all), n_ben_cal)
        atk_cal_idx = rs.choice(atk_idx_all, size=n_atk_cal, replace=False)

        held_out_mask = np.ones(len(y_te_mc), dtype=bool)
        held_out_mask[ben_slice] = False
        held_out_mask[atk_cal_idx] = False
        X_eval = X_te_all[held_out_mask]
        y_eval_bin = (y_te_mc[held_out_mask] != 0).astype(int)
        if X_eval.shape[0] == 0 or len(np.unique(y_eval_bin)) < 2:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: empty or single-class held-out evaluation set after slicing.\n")
            return

        X_train = np.vstack([X_attack_all, X_te_all[ben_train_idx]])
        y_train_bin = np.concatenate([np.ones(len(X_attack_all), dtype=int),
                                      np.zeros(len(ben_train_idx), dtype=int)])

        X_cal = np.vstack([X_te_all[atk_cal_idx], X_te_all[ben_cal_idx]])
        y_cal_bin = np.concatenate([np.ones(len(atk_cal_idx), dtype=int),
                                    np.zeros(len(ben_cal_idx), dtype=int)])

        pipe_b, mlp_b = fit_lr_then_mlp(
            X_train, y_train_bin,
            X_eval,  y_eval_bin,
            CFG["train"]["use_smote"], CFG["train"]["random_state"],
            CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
            CFG["train"]["mlp_hidden_units"]
        )
        if pipe_b is None:
            with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__SKIPPED.txt"), "w") as f:
                f.write("Skipped: training pipeline failed (single-class or pipeline error).\n")
            return

        save_artifact(pipe_b, os.path.join(RUN_DIR, "CIC_tiny_slice_pipe.joblib"))
        save_artifact(mlp_b,  os.path.join(RUN_DIR, "CIC_tiny_slice_mlp.joblib"))

        try:
            Z_eval = pipe_b.decision_function(X_eval).reshape(-1,1)
            Z_cal  = pipe_b.decision_function(X_cal).reshape(-1,1)
            prob_eval = mlp_b.predict_proba(Z_eval)[:,1]
            prob_cal  = mlp_b.predict_proba(Z_cal)[:,1]
        except Exception:
            prob_eval = pipe_b.predict_proba(X_eval)[:,1]
            prob_cal  = pipe_b.predict_proba(X_cal)[:,1]

        auc_unc = float(roc_auc_score(y_eval_bin, prob_eval))
        aupr_unc = float(average_precision_score(y_eval_bin, prob_eval))
        fpr90_unc, thr90_unc = fpr_at_dr(y_eval_bin, prob_eval, target_dr=0.90)
        fpr95_unc, thr95_unc = fpr_at_dr(y_eval_bin, prob_eval, target_dr=0.95)
        eval_binary(y_eval_bin, prob_eval, "CIC_IoMT__tiny_benign_slice", outdir)

        rows = []
        def _row(kind, auc, aupr, f90, t90, f95, t95):
            return {"scenario": f"CIC_tiny_slice__{kind}",
                    "roc_auc": auc, "aupr": aupr,
                    "fpr@dr=0.90": f90, "thr@dr=0.90": t90,
                    "fpr@dr=0.95": f95, "thr@dr=0.95": t95}
        rows.append(_row("uncalibrated", auc_unc, aupr_unc, fpr90_unc, thr90_unc, fpr95_unc, thr95_unc))

        for method in ["temperature", "isotonic"]:
            prob_eval_cal, meta = calibrate_scores(prob_cal, y_cal_bin, prob_eval, method=method)
            auc_c = float(roc_auc_score(y_eval_bin, prob_eval_cal))
            aupr_c = float(average_precision_score(y_eval_bin, prob_eval_cal))
            fpr90_c, thr90_c = fpr_at_dr(y_eval_bin, prob_eval_cal, target_dr=0.90)
            fpr95_c, thr95_c = fpr_at_dr(y_eval_bin, prob_eval_cal, target_dr=0.95)
            eval_binary(y_eval_bin, prob_eval_cal, f"CIC_IoMT__tiny_benign_slice__Calibrated({method})", outdir)
            with open(os.path.join(outdir, f"CIC_IoMT__tiny_benign_slice__Calibrated({method})__meta.json"), "w") as f:
                json.dump(meta, f, indent=2)
            rows.append(_row(f"Calibrated({method})", auc_c, aupr_c, fpr90_c, thr90_c, fpr95_c, thr95_c))

        Xt_tr_std = pipe_b.named_steps["scaler"].transform(
            pipe_b.named_steps["varth"].transform(
                pipe_b.named_steps["imputer"].transform(
                    pipe_b.named_steps["clean"].transform(X_train))))
        Xt_eval_std = pipe_b.named_steps["scaler"].transform(
            pipe_b.named_steps["varth"].transform(
                pipe_b.named_steps["imputer"].transform(
                    pipe_b.named_steps["clean"].transform(X_eval))))
        Xmin, Xmax = Xt_tr_std.min(axis=0), Xt_tr_std.max(axis=0)

        if Xt_eval_std.shape[0] > 200000:
            rs2 = np.random.RandomState(123)
            idx = rs2.choice(Xt_eval_std.shape[0], size=200000, replace=False)
            Xt_eval_std = Xt_eval_std[idx]; y_eval_small = y_eval_bin[idx]
        else:
            y_eval_small = y_eval_bin
        lr = pipe_b.named_steps["lr"]
        for eps in CFG["robust"]["eps"]:
            X_fgsm = fgsm(Xt_eval_std, y_eval_small, lr, eps)
            X_fgsm = clip_to_train_range(X_fgsm, pipe_b.named_steps["scaler"], Xmin, Xmax)
            z_fgsm = lr.decision_function(X_fgsm).reshape(-1,1)
            prob_fgsm = mlp_b.predict_proba(z_fgsm)[:,1] if mlp_b is not None else 1.0/(1.0+np.exp(-z_fgsm)).ravel()
            eval_binary(y_eval_small, prob_fgsm, f"CIC_IoMT__tiny_benign_slice__FGSM_eps={eps}", outdir)

            X_pgd = pgd(Xt_eval_std, y_eval_small, lr, eps, CFG['robust']['pgd_alpha'], CFG['robust']['pgd_steps'])
            X_pgd = clip_to_train_range(X_pgd, pipe_b.named_steps["scaler"], Xmin, Xmax)
            z_pgd = lr.decision_function(X_pgd).reshape(-1,1)
            prob_pgd = mlp_b.predict_proba(z_pgd)[:,1] if mlp_b is not None else 1.0/(1.0+np.exp(-z_pgd)).ravel()
            eval_binary(y_eval_small, prob_pgd, f"CIC_IoMT__tiny_benign_slice__PGD_eps={eps}", outdir)

        pd.DataFrame(rows).to_csv(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__summary.csv"), index=False)

        with open(os.path.join(RUN_DIR, "metrics_log.csv"), "a") as f:
            f.write(f"CIC_tiny_slice_uncalibrated,{auc_unc},{aupr_unc},{fpr90_unc},{fpr95_unc}\n")

    except Exception as e:
        with open(os.path.join(outdir, "CIC_IoMT__tiny_benign_slice__ERROR.txt"), "w") as f:
            f.write("Exception occurred during tiny-slice run:\n")
            f.write(str(e) + "\n\n" + traceback.format_exc())

In [ ]:
# -----------------------------------------------------------------------------------------------------
# CIC Option A (Publication-safe): strict Train/Validation/Test split with validation-only calibration
# -----------------------------------------------------------------------------------------------------
def _split_three_way(X, y, train_frac=0.60, val_frac=0.20, test_frac=0.20, seed=42):
    total = float(train_frac + val_frac + test_frac)
    if abs(total - 1.0) > 1e-6:
        raise ValueError(f"train_frac+val_frac+test_frac must sum to 1.0 (got {total})")
    # First split: Train vs Temp
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(
        X, y, test_size=(1.0 - train_frac), stratify=y, random_state=seed
    )
    # Second split: Val vs Test from Temp
    test_share_of_tmp = test_frac / (val_frac + test_frac)
    X_val, X_te, y_val, y_te = train_test_split(
        X_tmp, y_tmp, test_size=test_share_of_tmp, stratify=y_tmp, random_state=seed
    )
    return X_tr, y_tr, X_val, y_val, X_te, y_te

def _split_val_for_selection_and_calibration(X_val, y_val, val_cal_frac=0.50, seed=42):
    if not (0.0 < val_cal_frac < 1.0):
        raise ValueError("val_cal_frac must be in (0,1)")
    X_sel, X_cal, y_sel, y_cal = train_test_split(
        X_val, y_val, test_size=val_cal_frac, stratify=y_val, random_state=seed
    )
    return X_sel, y_sel, X_cal, y_cal

def _train_hybrid_lr_mlp(X_train, y_train, seed, C=1.0, hidden_units=64, scaler_name="standard"):
    pipe = get_lr_pipeline(CFG["train"]["use_smote"], seed, scaler_name=scaler_name, C=C)
    pipe.fit(X_train, y_train)
    try:
        Z_tr = pipe.decision_function(X_train).reshape(-1, 1)
    except Exception:
        p_tr = np.clip(pipe.predict_proba(X_train)[:, 1], 1e-7, 1-1e-7)
        Z_tr = np.log(p_tr / (1.0 - p_tr)).reshape(-1, 1)

    mlp = MLPClassifier(
        hidden_layer_sizes=(hidden_units,),
        alpha=1e-4,
        batch_size=CFG["train"]["batch_size"],
        learning_rate_init=1e-3,
        max_iter=CFG["train"]["max_epochs"],
        random_state=seed,
        early_stopping=True,
        n_iter_no_change=5,
        validation_fraction=0.1
    )
    mlp.fit(Z_tr, y_train)
    return pipe, mlp

def _predict_hybrid(pipe, mlp, X):
    try:
        Z = pipe.decision_function(X).reshape(-1, 1)
        return mlp.predict_proba(Z)[:, 1]
    except Exception:
        # Fallback: use LR probability if decision_function is unavailable
        return pipe.predict_proba(X)[:, 1]

def run_cic_optionA_threeway(cic_tr_df, cic_tr_feats, cic_te_df, cic_te_feats, outdir,
                             calib_methods=("temperature", "isotonic"),
                             seed=42):
    """
    Option A protocol (reviewer-safe):
      1) Combine CIC train+test into a single pool (original dataset).
      2) Strict three-way split: Train/Val/Test (default 60/20/20), stratified by binary label.
      3) Split Val into Val-selection and Val-calibration (disjoint).
      4) Hyperparam selection uses Val-selection only.
      5) Final model refit on Train + Val-selection; calibration fit on Val-calibration.
      6) Evaluate ONCE on Test with frozen model + frozen calibrator(s).
    """
    RUN_DIR = get_run_dir(outdir)
    os.makedirs(outdir, exist_ok=True)

    # Use only shared numeric features to avoid column drift.
    common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
    if len(common) == 0:
        with open(os.path.join(outdir, "CIC_IoMT__OptionA__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no shared numeric features between CIC train/test files.\n")
        return

    df_all = pd.concat([cic_tr_df, cic_te_df], ignore_index=True, sort=False)
    mc_col = detect_label_column(df_all, CFG["label_columns"]["multiclass_candidates"])
    bin_col = detect_label_column(df_all, CFG["label_columns"]["binary_candidates"])
    if (mc_col is None) and (bin_col is None):
        with open(os.path.join(outdir, "CIC_IoMT__OptionA__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no label column (binary or multiclass) found in CIC.\n")
        return

    y = build_binary_labels(df_all, bin_col, mc_col).astype(int)
    X = df_all[common].values

    # 60/20/20 split
    cf = CFG["cic_optionA"]
    X_tr, y_tr, X_val, y_val, X_te, y_te = _split_three_way(
        X, y,
        train_frac=cf["train_frac"],
        val_frac=cf["val_frac"],
        test_frac=cf["test_frac"],
        seed=seed
    )
    # Validation → selection + calibration
    X_val_sel, y_val_sel, X_val_cal, y_val_cal = _split_val_for_selection_and_calibration(
        X_val, y_val, val_cal_frac=cf["val_cal_frac"], seed=seed
    )

    # Hyperparameter selection (Val-selection only)
    best = {"auc": -np.inf, "C": None, "hidden": None, "pipe": None, "mlp": None}
    for C in cf.get("grid_C", [1.0]):
        for hidden in cf.get("grid_hidden", [CFG["train"]["mlp_hidden_units"]]):
            if len(np.unique(y_tr)) < 2:
                continue
            pipe, mlp = _train_hybrid_lr_mlp(X_tr, y_tr, seed=seed, C=C, hidden_units=hidden)
            p_sel = _predict_hybrid(pipe, mlp, X_val_sel)
            try:
                auc = roc_auc_score(y_val_sel, p_sel)
            except Exception:
                auc = -np.inf
            if auc > best["auc"]:
                best.update({"auc": float(auc), "C": float(C), "hidden": int(hidden), "pipe": pipe, "mlp": mlp})

    if best["pipe"] is None:
        with open(os.path.join(outdir, "CIC_IoMT__OptionA__SKIPPED.txt"), "w") as f:
            f.write("Skipped: hyperparameter selection failed (insufficient class variety).\n")
        return

    # Refit final model on Train + Val-selection (never touch Val-calibration or Test)
    X_fit = np.vstack([X_tr, X_val_sel])
    y_fit = np.concatenate([y_tr, y_val_sel])
    pipe_final, mlp_final = _train_hybrid_lr_mlp(
        X_fit, y_fit, seed=seed, C=best["C"], hidden_units=best["hidden"]
    )

    save_artifact(pipe_final, os.path.join(RUN_DIR, "CIC_OptionA_pipe.joblib"))
    save_artifact(mlp_final,  os.path.join(RUN_DIR, "CIC_OptionA_mlp.joblib"))
    with open(os.path.join(outdir, "CIC_IoMT__OptionA__selected_hparams.json"), "w") as f:
        json.dump({"C": best["C"], "hidden_units": best["hidden"], "val_sel_AUROC": best["auc"]}, f, indent=2)

    # Uncalibrated test evaluation
    p_test = _predict_hybrid(pipe_final, mlp_final, X_te)
    eval_binary(y_te, p_test, "CIC_IoMT__OptionA__uncalibrated", outdir)

    # Calibration probabilities from Val-calibration (never test)
    p_val_cal = _predict_hybrid(pipe_final, mlp_final, X_val_cal)

    rows = []
    def _row(kind, prob):
        auc = float(roc_auc_score(y_te, prob))
        aupr = float(average_precision_score(y_te, prob))
        f90, t90 = fpr_at_dr(y_te, prob, target_dr=0.90)
        f95, t95 = fpr_at_dr(y_te, prob, target_dr=0.95)
        return {"scenario": f"CIC_OptionA__{kind}",
                "roc_auc": auc, "aupr": aupr,
                "fpr@dr=0.90": float(f90) if f90==f90 else None, "thr@dr=0.90": float(t90) if t90 is not None else None,
                "fpr@dr=0.95": float(f95) if f95==f95 else None, "thr@dr=0.95": float(t95) if t95 is not None else None,
                "n_test": int(len(y_te)), "n_train": int(len(y_fit)),
                "n_val_sel": int(len(y_val_sel)), "n_val_cal": int(len(y_val_cal))}

    rows.append(_row("uncalibrated", p_test))

    for method in calib_methods:
        p_test_cal, meta = calibrate_scores(p_val_cal, y_val_cal.astype(int), p_test, method=method)
        eval_binary(y_te, p_test_cal, f"CIC_IoMT__OptionA__Calibrated({method})", outdir)
        with open(os.path.join(outdir, f"CIC_IoMT__OptionA__Calibrated({method})__meta.json"), "w") as f:
            json.dump(meta, f, indent=2)
        rows.append(_row(f"Calibrated({method})", p_test_cal))

    pd.DataFrame(rows).to_csv(os.path.join(outdir, "CIC_IoMT__OptionA__summary.csv"), index=False)

    # Append a compact log entry
    with open(os.path.join(RUN_DIR, "metrics_log.csv"), "a") as f:
        f.write(f"CIC_OptionA_uncalibrated,{rows[0]['roc_auc']},{rows[0]['aupr']},{rows[0]['fpr@dr=0.90']},{rows[0]['fpr@dr=0.95']}\n")

In [ ]:
# -----------------------------
#      Collapsed-binary
# -----------------------------
def run_in_domain_cic_collapsed(cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                                cic_te_df, cic_te_feats, cic_te_mc_col, outdir, calib_method="temperature"):
    common = list(sorted(set(cic_tr_feats).intersection(set(cic_te_feats))))
    if len(common) == 0:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no shared numeric features between CIC_train and CIC_test.\n")
        return
    Xtr = cic_tr_df[common].values
    Xte_full = cic_te_df[common].values
    ym_tr = cic_tr_df[cic_tr_mc_col].astype(int).values if cic_tr_mc_col else None
    ym_te_full = cic_te_df[cic_te_mc_col].astype(int).values if cic_te_mc_col else None
    if ym_tr is None or ym_te_full is None or len(np.unique(ym_tr)) < 2:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: multiclass labels missing or single-class in CIC_train/CIC_test.\n")
        return

    pipe_m = SkPipeline([
    ("clean", SafeNaNDropper()),
    ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, n_jobs=-1))
    ])
    pipe_m.fit(Xtr, ym_tr)

    X_cal, X_eval, y_cal, y_eval = train_test_split(Xte_full, ym_te_full, test_size=1.0-CFG["calibration"]["cic_calib_frac"],
                                                    stratify=ym_te_full, random_state=CFG["train"]["random_state"])
    probas_cal = pipe_m.predict_proba(X_cal)
    probas_eval = pipe_m.predict_proba(X_eval)
    ben_mask_cal = (y_cal == 0)
    if ben_mask_cal.sum() == 0:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write("Skipped: no benign in CIC_test calibration split; cannot align benign column.\n")
        return
    mean_probs_on_ben = probas_cal[ben_mask_cal].mean(axis=0)
    benign_idx_aligned = int(np.argmax(mean_probs_on_ben))
    if mean_probs_on_ben[benign_idx_aligned] < 0.25:
        with open(os.path.join(outdir, "CIC_IoMT__train_to_test__binary_from_mc__SKIPPED.txt"), "w") as f:
            f.write(f"Skipped: aligned 'benign' column probability on benign-cal < 0.25 ({mean_probs_on_ben[benign_idx_aligned]:.3f}).\n")
        return
    s_attack_cal  = 1.0 - np.clip(probas_cal[:,  benign_idx_aligned], 1e-6, 1-1e-6)
    s_attack_eval = 1.0 - np.clip(probas_eval[:, benign_idx_aligned], 1e-6, 1-1e-6)
    auc_cal = roc_auc_score((y_cal != 0).astype(int), s_attack_cal)
    if auc_cal < 0.5:
        s_attack_cal  = 1.0 - s_attack_cal
        s_attack_eval = 1.0 - s_attack_eval
    eval_binary((y_eval != 0).astype(int), s_attack_eval, "CIC_IoMT__train_to_test__binary_from_mc", outdir)
    s_eval_cal, meta = calibrate_scores(s_attack_cal, (y_cal != 0).astype(int), s_attack_eval, method=calib_method)
    eval_binary((y_eval != 0).astype(int), s_eval_cal, f"CIC_IoMT__train_to_test__binary_from_mc__Calibrated({calib_method})", outdir)
    with open(os.path.join(outdir, f"CIC_IoMT__train_to_test__binary_from_mc__Calibrated({calib_method})__meta.json"), "w") as f:
        json.dump({"benign_alignment": {"column_index": benign_idx_aligned,
                                        "mean_prob_on_benign_cal": float(mean_probs_on_ben[benign_idx_aligned])},
                   **meta}, f, indent=2)

In [ ]:
# -----------------------
#      Cross-domain
# -----------------------
def run_cross_domain(nf_df, nf_feats, nf_bin_col, nf_mc_col,
                     cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                     cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                     outdir, automap_min=5):
    RUN_DIR = get_run_dir(outdir)

    nf_df2, cic_te_df2, common_nf_cic, audit1 = apply_automap_and_rename(nf_df, nf_feats, cic_te_df, cic_te_feats, outdir, "NF_to_CIC")
    if len(common_nf_cic) < automap_min:
        with open(os.path.join(outdir, "NF_to_CIC__xfer__SKIPPED.txt"), "w") as f:
            f.write(f"Skipped: shared features {len(common_nf_cic)} < {automap_min}\n")
    else:
        Xs = nf_df2[common_nf_cic].values
        Xt = cic_te_df2[common_nf_cic].values
        ys = build_binary_labels(nf_df2, nf_bin_col, nf_mc_col)
        yt = build_binary_labels(cic_te_df2, cic_te_bin_col, cic_te_mc_col)
        pipe, mlp = fit_lr_then_mlp(Xs, ys, Xt, np.zeros(len(Xt)),
                                    CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                    CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                    CFG["train"]["mlp_hidden_units"])
        if pipe is not None and mlp is not None:
            save_artifact(pipe, os.path.join(RUN_DIR, "XFER_NF_to_CIC_pipe.joblib"))
            save_artifact(mlp,  os.path.join(RUN_DIR, "XFER_NF_to_CIC_mlp.joblib"))
        prob_pos = None
        if pipe is not None:
            try:
                Zt = pipe.decision_function(Xt).reshape(-1,1)
            except Exception:
                Zt = np.log(np.clip(pipe.predict_proba(Xt), 1e-7, 1-1e-7)).reshape(-1,1)
            prob_pos = mlp.predict_proba(Zt)[:,1] if mlp is not None else pipe.predict_proba(Xt)[:,1]
        eval_binary(yt, prob_pos, "NF_ToN_IoT__to__CIC_IoMT_test__binary_xfer", outdir)
        if pipe is not None:
            Xt_std = pipe.named_steps["scaler"].transform(
                pipe.named_steps["varth"].transform(
                    pipe.named_steps["imputer"].transform(
                        pipe.named_steps["clean"].transform(Xt))))
            Xs_std = pipe.named_steps["scaler"].transform(
                pipe.named_steps["varth"].transform(
                    pipe.named_steps["imputer"].transform(
                        pipe.named_steps["clean"].transform(Xs))))
            Xmin, Xmax = Xs_std.min(axis=0), Xs_std.max(axis=0)
            lr = pipe.named_steps["lr"]
            for eps in CFG["robust"]["eps"]:
                Xt_fgsm = fgsm(Xt_std, yt, lr, eps)
                Xt_fgsm = clip_to_train_range(Xt_fgsm, pipe.named_steps["scaler"], Xmin, Xmax)
                z_fgsm = lr.decision_function(Xt_fgsm).reshape(-1,1)
                prob_fgsm = mlp.predict_proba(z_fgsm)[:,1] if mlp is not None else 1.0/(1.0+np.exp(-z_fgsm)).ravel()
                eval_binary(yt, prob_fgsm, f"NF_ToN_IoT__to__CIC_IoMT_test__FGSM_eps={eps}", outdir)

                Xt_pgd = pgd(Xt_std, yt, lr, eps, CFG["robust"]["pgd_alpha"], CFG["robust"]["pgd_steps"])
                Xt_pgd = clip_to_train_range(Xt_pgd, pipe.named_steps["scaler"], Xmin, Xmax)
                z_pgd = lr.decision_function(Xt_pgd).reshape(-1,1)
                prob_pgd = mlp.predict_proba(z_pgd)[:,1] if mlp is not None else 1.0/(1.0+np.exp(-z_pgd)).ravel()
                eval_binary(yt, prob_pgd, f"NF_ToN_IoT__to__CIC_IoMT_test__PGD_eps={eps}", outdir)

    cic_tr_df2, nf_df2b, common_cic_nf, audit2 = apply_automap_and_rename(cic_tr_df, cic_tr_feats, nf_df, nf_feats, outdir, "CIC_to_NF")
    if len(common_cic_nf) < automap_min:
        with open(os.path.join(outdir, "CIC_to_NF__xfer__SKIPPED.txt"), "w") as f:
            f.write(f"Skipped: shared features {len(common_cic_nf)} < {automap_min}\n")
    else:
        Xs2 = cic_tr_df2[common_cic_nf].values
        Xt2 = nf_df2b[common_cic_nf].values
        ys2 = build_binary_labels(cic_tr_df2, cic_tr_bin_col, cic_tr_mc_col)
        yt2 = build_binary_labels(nf_df2b, nf_bin_col, nf_mc_col)
        pipe2, mlp2 = fit_lr_then_mlp(Xs2, ys2, Xt2, np.zeros(len(Xt2)),
                                      CFG["train"]["use_smote"], CFG["train"]["random_state"],
                                      CFG["train"]["max_epochs"], CFG["train"]["batch_size"],
                                      CFG["train"]["mlp_hidden_units"])
        if pipe2 is not None and mlp2 is not None:
            save_artifact(pipe2, os.path.join(RUN_DIR, "XFER_CIC_to_NF_pipe.joblib"))
            save_artifact(mlp2,  os.path.join(RUN_DIR, "XFER_CIC_to_NF_mlp.joblib"))
        prob_pos2=None
        if pipe2 is not None:
            try:
                Zt2 = pipe2.decision_function(Xt2).reshape(-1,1)
            except Exception:
                Zt2 = np.log(np.clip(pipe2.predict_proba(Xt2), 1e-7, 1-1e-7)).reshape(-1,1)
            prob_pos2 = mlp2.predict_proba(Zt2)[:,1] if mlp2 is not None else pipe2.predict_proba(Xt2)[:,1]
        eval_binary(yt2, prob_pos2, "CIC_IoMT_train__to__NF_ToN_IoT__binary_xfer", outdir)

In [ ]:
# ======================================================================================
# ========= CLEAN ABLATION (LR, MLP, LR→MLP uncal./Temp/Isot.), SEED-AVERAGED) =========
# ======================================================================================
SEEDS_FOR_ABLATION = [0,1,2,3,4]

def _train_lr(Xtr, ytr, seed):
    return LogisticRegression(C=1.0, max_iter=2000, solver="lbfgs",
                              class_weight="balanced", n_jobs=-1,
                              random_state=seed).fit(Xtr, ytr)

def _train_mlp(Xtr, ytr, seed, hidden=CFG["train"]["mlp_hidden_units"], max_epochs=CFG["train"]["max_epochs"], batch=CFG["train"]["batch_size"]):
    return MLPClassifier(hidden_layer_sizes=(hidden,), activation="relu",
                         learning_rate_init=1e-3, alpha=1e-4,
                         batch_size=batch, max_iter=max_epochs,
                         early_stopping=True, n_iter_no_change=5,
                         validation_fraction=0.1, random_state=seed).fit(Xtr, ytr)

def _hybrid_lr_mlp_preds(Xtr, ytr, Xte, seed):
    lr = _train_lr(Xtr, ytr, seed)
    try:
        z_tr = lr.decision_function(Xtr).reshape(-1,1)
        z_te = lr.decision_function(Xte).reshape(-1,1)
    except Exception:
        p_tr = np.clip(lr.predict_proba(Xtr), 1e-7, 1-1e-7)
        p_te = np.clip(lr.predict_proba(Xte), 1e-7, 1-1e-7)
        z_tr = np.log(p_tr[:,1] / p_tr[:,0]).reshape(-1,1)
        z_te = np.log(p_te[:,1] / p_te[:,0]).reshape(-1,1)
    mlp = _train_mlp(z_tr, ytr, seed)
    prob_uncal = mlp.predict_proba(z_te)[:,1]
    return lr, mlp, z_tr, z_te, prob_uncal

def _metrics_binary(y_true, prob, n_bins=15):
    auroc = roc_auc_score(y_true, prob)
    aupr  = average_precision_score(y_true, prob)
    fpr95, _ = fpr_at_dr(y_true, prob, target_dr=0.95)
    prob2 = np.vstack([1-prob, prob]).T
    ece = expected_calibration_error(y_true.astype(int), prob2, n_bins=n_bins)
    nll = log_loss(y_true, np.clip(prob2, 1e-12, 1-1e-12), labels=[0,1])
    return dict(AUROC=auroc, AUPR=aupr, FPR95=fpr95, ECE=ece, NLL=nll)

def _eval_variant_set(dataset_name, X, y, seeds):
    rows = []
    for seed in seeds:
        Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
        sc = StandardScaler().fit(Xtr)
        Xtrs, Xtes = sc.transform(Xtr), sc.transform(Xte)

        # LR
        lr = _train_lr(Xtrs, ytr, seed)
        plr = lr.predict_proba(Xtes)[:,1]
        rows.append(dict(Dataset=dataset_name, Model="LR", Seed=seed, **_metrics_binary(yte, plr)))

        # MLP
        mlp = _train_mlp(Xtrs, ytr, seed)
        pmlp = mlp.predict_proba(Xtes)[:,1]
        rows.append(dict(Dataset=dataset_name, Model="MLP", Seed=seed, **_metrics_binary(yte, pmlp)))

        # Hybrid LR→MLP (uncal.)
        lr_h, mlp_h, z_tr, z_te, p_uncal = _hybrid_lr_mlp_preds(Xtrs, ytr, Xtes, seed)
        rows.append(dict(Dataset=dataset_name, Model="LR→MLP (uncal.)", Seed=seed, **_metrics_binary(yte, p_uncal)))

        # Calibration split from TRAIN (10%)
        Xtr0, Xcal, ytr0, ycal = train_test_split(Xtrs, ytr, test_size=0.10, stratify=ytr, random_state=seed)
        lr_h2, mlp_h2, z_tr0, z_te2, p_uncal2 = _hybrid_lr_mlp_preds(Xtr0, ytr0, Xtes, seed)
        # Cal logits->prob on CAL
        try:
            z_cal = lr_h2.decision_function(Xcal).reshape(-1,1)
            p_cal = mlp_h2.predict_proba(z_cal)[:,1]
        except Exception:
            p_cal = mlp_h2.predict_proba(Xcal)[:,1]

        # Temp
        p_temp, _ = calibrate_scores(p_cal, (ycal==1).astype(int), p_uncal2, method="temperature")
        rows.append(dict(Dataset=dataset_name, Model="LR→MLP (Temp)", Seed=seed, **_metrics_binary(yte, p_temp)))
        # Isot.
        p_iso, _  = calibrate_scores(p_cal, (ycal==1).astype(int), p_uncal2, method="isotonic")
        rows.append(dict(Dataset=dataset_name, Model="LR→MLP (Isot.)", Seed=seed, **_metrics_binary(yte, p_iso)))
    return pd.DataFrame(rows)

def build_clean_ablation_tables(nf_df, nf_feats, nf_bin_col,
                                cic_tr_df, cic_tr_feats, cic_te_df, cic_te_feats,
                                outdir):
    os.makedirs(outdir, exist_ok=True)
    # NF binary
    y_nf = build_binary_labels(nf_df, nf_bin_col, None)
    X_nf = nf_df[nf_feats].values
    df_nf = _eval_variant_set("NF-ToN-IoT", X_nf, y_nf, SEEDS_FOR_ABLATION)

    # CIC binary from native multiclass (combined pool; consistent with Option A)
    common = sorted(set(cic_tr_feats).intersection(set(cic_te_feats)))
    df_cic_all = pd.concat([cic_tr_df, cic_te_df], ignore_index=True, sort=False)
    mc = detect_label_column(df_cic_all, CFG["label_columns"]["multiclass_candidates"])
    if mc is None:
        raise ValueError("CIC multiclass label column not found for ablation table.")
    X_cic = df_cic_all[common].values
    y_cic = (df_cic_all[mc].astype(int).values != 0).astype(int)
    df_cic = _eval_variant_set("CIC-IoMT (WiFi--MQTT)", X_cic, y_cic, SEEDS_FOR_ABLATION)

    df_all = pd.concat([df_nf, df_cic], ignore_index=True)

    abl = (df_all.groupby(["Dataset","Model"])
                 .agg(AUROC=("AUROC","mean"),
                      AUPR=("AUPR","mean"),
                      FPR95=("FPR95","mean"),
                      ECE=("ECE","mean"),
                      NLL=("NLL","mean"))
                 .reset_index())

    for c in ["AUROC","AUPR","FPR95","ECE","NLL"]:
        abl[c] = abl[c].round(2)

    order = ["LR","MLP","LR→MLP (uncal.)","LR→MLP (Temp)","LR→MLP (Isot.)"]
    abl["rank"] = abl["Model"].apply(lambda m: order.index(m) if m in order else 999)
    abl = abl.sort_values(["Dataset","rank"]).drop(columns=["rank"])

    csv_path = os.path.join(outdir, "clean_ablation_seedavg.csv")
    abl.to_csv(csv_path, index=False)

    lines = [f"{r.Dataset} & {r.Model} & {r.AUROC:.2f} & {r.AUPR:.2f} & {r.FPR95:.2f} & {r.ECE:.2f} & {r.NLL:.2f} \\\\"
             for r in abl.itertuples()]
    latex = r"""\begin{table}[!t]
\centering
\caption{Clean-condition ablation (seed-averaged). LR and MLP alone vs the hybrid LR$\rightarrow$MLP with/without calibration.}
\label{tab:ablation-clean}
\begin{tabular}{lcccccc}
\toprule
Dataset & Model & AUROC $\uparrow$ & AUPR (macro) $\uparrow$ & FPR@95\%DR $\downarrow$ & ECE $\downarrow$ & NLL $\downarrow$ \\
\midrule
""" + "\n".join(lines) + r"""
\bottomrule
\end{tabular}
\end{table}
"""
    tex_path = os.path.join(outdir, "clean_ablation_seedavg.tex")
    with open(tex_path, "w") as f:
        f.write(latex)
    print(latex)

In [ ]:
# -----------------
#    ZIP helper
# -----------------
def zip_outputs(outdir, zip_path):
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(outdir):
            for f in files:
                fp = os.path.join(root, f)
                arc = os.path.relpath(fp, start=outdir)
                zf.write(fp, arc)
    return zip_path

In [ ]:
# -----------------
#       Main
# -----------------
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--grid", action="store_true", help="(unused here)")
    parser.add_argument("--adv-train-eps", type=float, default=0.0)
    parser.add_argument("--adv-train-frac", type=float, default=0.3)
    parser.add_argument("--cic-calib", type=str, choices=["temperature","isotonic"], default="temperature")
    parser.add_argument("--automap-min", type=int, default=5, help="Minimum shared features for cross-domain metrics")
    parser.add_argument("--cic-protocol", type=str, choices=["optionA","legacy","both"], default="optionA",
                        help="CIC evaluation protocol: 'optionA' is publication-safe (strict Train/Val/Test + val-only calibration).")
    parser.add_argument("--legacy-cic-collapsed", action="store_true",
                        help="Run legacy CIC train->test collapsed-binary experiment (uses CIC_test for calibration; not publication-safe).")
    parser.add_argument("--legacy-cic-tiny-slice", action="store_true",
                        help="Run legacy CIC tiny-slice experiment (uses CIC_test flows for training/calibration; not publication-safe).")
    parser.add_argument("--no-cic-tiny-slice", action="store_true", help="Disable the CIC tiny benign slice experiment (enabled by default).")
    parser.add_argument("--cic-slice-frac", type=float, default=0.015, help="Fraction of CIC_test benign to use for training+calibration (default 1.5%).")
    parser.add_argument("--cic-slice-seed", type=int, default=42, help="Random seed for benign-slice sampling.")
    parser.add_argument("--zip", action="store_true", help="Also compress outputs to a ZIP bundle.")
    args, _ = parser.parse_known_args()

    outdir = CFG["paths"]["outdir"]
    os.makedirs(outdir, exist_ok=True)
    os.makedirs(get_run_dir(outdir), exist_ok=True)

    nf_df, nf_feats, nf_bin_col, nf_mc_col = load_dataset(CFG["paths"]["nf_csv"])
    cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col = load_dataset(CFG["paths"]["cic_train_csv"])
    cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col = load_dataset(CFG["paths"]["cic_test_csv"])

    with open(os.path.join(outdir, "hyperparams.json"), "w") as f:
        json.dump({
            "random_state": CFG["train"]["random_state"],
            "use_smote": CFG["train"]["use_smote"],
            "mlp_hidden_units": CFG["train"]["mlp_hidden_units"],
            "max_epochs": CFG["train"]["max_epochs"],
            "batch_size": CFG["train"]["batch_size"],
            "target_drs": CFG["metrics"]["target_drs"],
            "robust_eps": CFG["robust"]["eps"],
            "pgd_steps": CFG["robust"]["pgd_steps"],
            "pgd_alpha": CFG["robust"]["pgd_alpha"],
            "cic_calib_frac": CFG["calibration"]["cic_calib_frac"],
            "cic_calib_method": args.cic_calib,
            "automap_min": args.automap_min,
            "automap_threshold": CFG["automap"]["similarity_threshold"],
            "automap_max_pairs": CFG["automap"]["max_pairs"],
            "cic_slice_frac": args.cic_slice_frac,
            "cic_slice_seed": args.cic_slice_seed
        }, f, indent=2)

    # Existing evaluations
    run_in_domain_nf(nf_df, nf_feats, nf_bin_col, nf_mc_col, outdir)

    # -----------------------------
    #      CIC evaluations
    # -----------------------------
    if args.cic_protocol in ["optionA", "both"]:
        # Publication-safe protocol: strict Train/Val/Test + validation-only calibration
        run_cic_optionA_threeway(
            cic_tr_df, cic_tr_feats,
            cic_te_df, cic_te_feats,
            outdir,
            calib_methods=("temperature", "isotonic"),
            seed=CFG["train"]["random_state"]
        )

    if args.cic_protocol in ["legacy", "both"]:
        # Legacy/diagnostic protocols (NOT publication-safe if they touch CIC_test for calibration/training)
        if cic_tr_mc_col and cic_te_mc_col:
            run_in_domain_cic_native_mc(
                cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                cic_te_df, cic_te_feats, cic_te_mc_col, outdir
            )
            if args.legacy_cic_collapsed:
                run_in_domain_cic_collapsed(
                    cic_tr_df, cic_tr_feats, cic_tr_mc_col,
                    cic_te_df, cic_te_feats, cic_te_mc_col,
                    outdir, calib_method=args.cic_calib
                )

        if args.legacy_cic_tiny_slice:
            run_cic_with_tiny_benign_slice(
                cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                outdir, slice_frac=args.cic_slice_frac, seed=args.cic_slice_seed
            )


    run_cross_domain(nf_df, nf_feats, nf_bin_col, nf_mc_col,
                     cic_tr_df, cic_tr_feats, cic_tr_bin_col, cic_tr_mc_col,
                     cic_te_df, cic_te_feats, cic_te_bin_col, cic_te_mc_col,
                     outdir, automap_min=args.automap_min)

    # >>> NEW: Seed-averaged CLEAN ablation table (LR, MLP, LR→MLP uncal./Temp/Isot.)
    build_clean_ablation_tables(
        nf_df, nf_feats, nf_bin_col,
        cic_tr_df, cic_tr_feats, cic_te_df, cic_te_feats,
        outdir=os.path.join(CFG["paths"]["outdir"], "paper_exports")
    )

    if args.zip:
        zip_path = os.path.join(Path(outdir).parent, "outputs_bundle.zip")
        zip_outputs(outdir, zip_path)
        print(f"[OK] Wrote ZIP bundle → {zip_path}")

if __name__ == "__main__":
    main()